In [ ]:
#import pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from point_transformer_pytorch import PointTransformerLayer

In [ ]:
# input (I, 13)
# input neighbours (J, 13)
class LrgNet(nn.Module):
    def __init__(self, batch_size=32,
                 seq_len=1, # Number of region growing steps unrolled
                 num_inlier_points=512,
                 num_neighbour_points=512,
                 feature_dim=13):
        super(LrgNet, self).__init__()
        self.B1 = [128, 128]
        self.B2 = [128, 256, 512, 1024]
        self.B3 = [512, 256, 128]
        
        # Initial up-dimension layers for inlier and neighbour points (to be updated)
        up_dim_inlier_b1 = nn.Linear(feature_dim, self.B1[0])
        up_dim_neighbour_b1 = nn.Linear(feature_dim, self.B1[0])

        # Point Transformer Layers for inlier set
        for i in range(len(self.B1)-1):
            inlier_layer = PointTransformerLayer(dim=self.B1[i], dim_out=self.B1[i+1], pos_mlp_hidden_dim=32, attn_mlp_hidden_dim=32)
            neighbour_layer = PointTransformerLayer(dim=self.B1[i], dim_out=self.B1[i+1], pos_mlp_hidden_dim=32, attn_mlp_hidden_dim=32)
            setattr(self, 'B1_inlier_' + str(i), inlier_layer)
            setattr(self, 'B1_neighbour_' + str(i), neighbour_layer)
        
        # Point Transformer Layers for outlier set
        for i in range(len(self.B2)-1):
            inlier_layer = PointTransformerLayer(dim=self.B2[i], dim_out=self.B2[i+1], pos_mlp_hidden_dim=32, attn_mlp_hidden_dim=32)
            neighbour_layer = PointTransformerLayer(dim=self.B2[i], dim_out=self.B2[i+1], pos_mlp_hidden_dim=32, attn_mlp_hidden_dim=32)
            setattr(self, 'B1_inlier_' + str(i), inlier_layer)
            setattr(self, 'B1_neighbour_' + str(i), neighbour_layer)



        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Assuming input images are 32x32
        self.fc2 = nn.Linear(128, 10)  # Assuming 10 classes for output

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
import torch
from point_transformer_pytorch import PointTransformerLayer

attn = PointTransformerLayer(
    dim = 128,
    pos_mlp_hidden_dim = 64,
    attn_mlp_hidden_mult = 4,
)

feats = torch.randn(1, 16, 128)
pos = torch.randn(1, 16, 3)
mask = torch.ones(1, 16).bool()

out = attn(feats, pos, mask = mask) # (1, 16, 128)

# print output shape
out.shape

TypeError: __init__() got an unexpected keyword argument 'dim_out'